# LangGraph と AgentCore Memory Hooks（長期記憶）

## はじめに

このノートブックでは、LangGraph フレームワークを使用して Amazon Bedrock AgentCore Memory 機能を会話型 AI エージェントに統合する方法を紹介します。**長期記憶**の保持に焦点を当て、エージェントが過去のインタラクションからユーザーの好み、食事制限、コンテキスト情報を抽出して想起できるようにします。

## チュートリアルの詳細

| 項目               | 詳細                                                                             |
|:-------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ | 長期記憶型会話                                                                   |
| エージェントユースケース | 栄養アシスタント                                                              |
| エージェントフレームワーク | LangGraph                                                                    |
| LLM モデル          | Anthropic Claude Haiku 4.5                                                      |
| チュートリアルコンポーネント | AgentCore 長期 Memory、カスタム Memory 戦略、Pre/Post モデルフック          |
| 難易度              | 中級                                                                             |

学習内容：
- UserPreference カスタムオーバーライド戦略を使用した AgentCore Memory の作成
- 自動メモリ保存と取得のための pre/post モデルフックの実装
- セッション間でユーザーの好みを記憶する栄養アシスタントの構築
- 関連するユーザーコンテキストを取得するためのセマンティック検索の使用
- カスタムメモリ抽出と統合プロンプトの設定

### シナリオの背景

この例では、食事制限、お気に入りの食べ物、調理の好み、健康目標などのユーザーコンテキストを複数の会話にわたって記憶できる**栄養アシスタント**を作成します。エージェントは会話からユーザーの好みを自動的に抽出して保存し、将来のインタラクションで関連するコンテキストを取得してパーソナライズされた栄養アドバイスを提供します。

## アーキテクチャ

<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件

- Python 3.10以上
- 適切な権限を持つ AWS アカウント
- AgentCore Memory の適切な権限を持つ AWS IAM ロール
- Amazon Bedrock モデルへのアクセス

環境のセットアップから始めましょう！

In [ ]:
# Install necessary libraries from https://github.com/langchain-ai/langchain-aws
%pip install -qr requirements.txt

In [ ]:
import os
import logging

# Import LangGraph and LangChain components
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnableConfig
from langgraph.store.base import BaseStore
import uuid


region = os.getenv('AWS_REGION', 'us-east-1')
logging.getLogger("math-agent").setLevel(logging.DEBUG)

In [ ]:
# Import the AgentCoreMemoryStore that we will use as a store
from langgraph_checkpoint_aws import (
    AgentCoreMemoryStore
)

# For this example, we will just use an InMemorySaver to save context.
# In production, we highly recommend the AgentCoreMemorySaver as a checkpointer which works seamlessly alongside the memory store
#from langgraph_checkpoint_aws import AgentCoreMemorySaver
from langgraph.checkpoint.memory import InMemorySaver
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

from custom_memory_prompts import consolidation_prompt, extraction_prompt

In [ ]:
memory_name = "NutritionAssistant"
client = MemoryClient(region_name=region)
MODEL_ID = "global.anthropic.claude-haiku-4-5-20251001-v1:0"

memory = client.create_or_get_memory(
    name=memory_name,
    description="Nutrition assistant",
    memory_execution_role_arn="arn:aws:iam::YOUR_ACCOUNT:role/YOUR_ROLE", # Please provide a role with a valid trust policy
    strategies=[
        {
            StrategyType.CUSTOM.value: {
                "name": "NutritionPreferences",
                "description": "Captures customer food preferences and behavior",
                "namespaces": ["/{actorId}/preferences"],
                "configuration": {
                    "userPreferenceOverride": {
                        "extraction": {
                            "appendToPrompt": extraction_prompt,
                            "modelId": MODEL_ID,
                        },
                        "consolidation": {
                            "appendToPrompt": consolidation_prompt,
                            "modelId": MODEL_ID,
                        }
                    }
                }
            }
        },
    ]
)
memory_id = memory["id"]

### Memory 設定の概要

AgentCore Memory のセットアップには以下が含まれます：

- **Custom Strategy**: 会話から栄養の好みを抽出
- **Namespaces**: ユーザーごとにメモリを整理（`{actorId}/preferences`）
- **Custom Prompts**: 食べ物の好み用の特殊化された抽出と統合ロジック
- **Model Integration**: メモリ処理に Claude 3.7 Sonnet を使用

メモリシステムは、一時的または無関係な情報をフィルタリングしながら、永続的なユーザーの好みを抽出するために会話を自動的に処理します。

## ステップ 3: Memory Store と LLM の初期化

次に、AgentCore Memory Store と言語モデルを初期化します。

In [ ]:
# Initialize the store to enable long term memory saving and retrieval
store = AgentCoreMemoryStore(memory_id=memory_id, region_name=region)

# Initialize Bedrock LLM
llm = init_chat_model(MODEL_ID, model_provider="bedrock_converse", region_name=region)

## ステップ 4: Memory Hooks の実装

メモリの保存と取得を自動的に処理する pre および post モデルフックを作成します：

- **Pre-model hook**: 関連するユーザーの好み（セマンティック検索に基づく）を取得し、LLM 呼び出し前にコンテキストを追加
- **Post-model hook**: 長期記憶抽出のために会話メッセージを保存

### Memory 処理の仕組み

1. メッセージは actor_id と session_id とともに AgentCore Memory に保存されます
2. カスタム戦略が会話を処理して栄養の好みを抽出します
3. 抽出された好みは `{actorId}/preferences` 名前空間に保存されます
4. 将来の会話でコンテキストのために関連する好みを検索・取得できます

**注意**: LangChain のメッセージタイプは、ストアによって内部で AgentCore Memory のメッセージタイプに変換されるため、長期記憶に適切に抽出できます。

In [ ]:
def pre_model_hook(state, config: RunnableConfig, *, store: BaseStore):
    """Hook that runs pre-LLM invocation to save the latest human message"""
    actor_id = config["configurable"]["actor_id"]
    thread_id = config["configurable"]["thread_id"]
    # Saving the message to the actor and session combination that we get at runtime
    namespace = (actor_id, thread_id)
    
    messages = state.get("messages", [])
    # Save the last human message we see before LLM invocation
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            store.put(namespace, str(uuid.uuid4()), {"message": msg})
            break
    # Retrieve user preferences based on the last message and append to state
    user_preferences_namespace = (actor_id, "preferences")
    preferences = store.search(user_preferences_namespace, query=msg.content, limit=5)
    
    # Construct another AI message to add context before the current message
    if preferences:
        context_items = [pref.value for pref in preferences]
        context_message = AIMessage(
            content=f"[User Context: {', '.join(str(item) for item in context_items)}]"
        )
        # Insert the context message before the last human message
        return {"messages": messages[:-1] + [context_message, messages[-1]]}
    
    return {"llm_input_messages": messages}

def post_model_hook(state, config: RunnableConfig, *, store: BaseStore):
    """Hook that runs post-LLM invocation to save the latest human message"""
    actor_id = config["configurable"]["actor_id"]
    thread_id = config["configurable"]["thread_id"]

    # Saving the message to the actor and session combination that we get at runtime
    namespace = (actor_id, thread_id)
    
    messages = state.get("messages", [])
    # Save the LLMs response to AgentCore Memory
    for msg in reversed(messages):
        if isinstance(msg, AIMessage):
            store.put(namespace, str(uuid.uuid4()), {"message": msg})
            break
    
    return {"messages": messages}

## ステップ 5: LangGraph Agent の作成

メモリフックを統合した LangGraph の `create_react_agent` を使用して栄養アシスタントエージェントを作成します。ツールノードには長期記憶取得ツールのみが含まれ、pre および post モデルフックが引数として指定されます。

**注意**: カスタムエージェント実装の場合、Store とツールは任意のワークフローでこのパターンに従って必要に応じて設定できます。Pre/post モデルフックを使用することも、会話全体を最後に保存することもできます。

In [ ]:
graph = create_react_agent(
    llm,
    store=store,
    tools=[], # No additional tools needed for this example
    checkpointer=InMemorySaver(), # For conversation state management
    pre_model_hook=pre_model_hook, # Retrieves user preferences before LLM call
    post_model_hook=post_model_hook  # Saves conversation after LLM response
)

## ステップ 6: Agent ランタイムの設定

ユーザーとセッションの一意の識別子でエージェントを設定する必要があります。これらの ID はメモリの整理と取得に不可欠です。

### Graph の入力
`inputs` 引数として最新のユーザーメッセージを渡すだけで済みます。他の状態変数も含めることができますが、シンプルな `create_react_agent` では messages のみが必要です。

### LangGraph RuntimeConfig
LangGraph では、config は呼び出し時に必要な属性（ユーザー ID やセッション ID など）を含む `RuntimeConfig` です。`AgentCoreMemorySaver` の場合、`thread_id` と `actor_id` を config に設定する必要があります。例えば、AgentCore の呼び出しエンドポイントは、呼び出し元の ID またはユーザー ID に基づいてこれを割り当てることができます。追加の[ドキュメントはこちら](https://langchain-ai.github.io/langgraphjs/how-tos/configuration/)で確認できます。



In [ ]:
actor_id = "user-1"
config = {
    "configurable": {
        "thread_id": "session-1", # REQUIRED: This maps to Bedrock AgentCore session_id under the hood
        "actor_id": actor_id, # REQUIRED: This maps to Bedrock AgentCore actor_id under the hood
    }
}

## ステップ 7: Agent のテスト

食べ物の好みについての会話で栄養アシスタントをテストしましょう。エージェントは将来の使用のためにユーザーの好みを自動的に抽出して保存します。

In [ ]:
# Helper function to pretty print agent output while running
def run_agent(query: str, config: RunnableConfig):
    printed_ids = set()
    events = graph.stream(
        {"messages": [{"role": "user", "content": query}]},
        config,
        stream_mode="values",
    )
    for event in events:
        if "messages" in event:
            for msg in event["messages"]:
                # Check if we've already printed this message
                if id(msg) not in printed_ids:
                    msg.pretty_print()
                    printed_ids.add(id(msg))


prompt = """
Hey there! Im cooking one of my favorite meals tonight, salmon with rice and veggies (healthy). Has
great macros for my weightlifting competition that is coming up. What can I add to this dish to make it taste better
and also improve the protein and vitamins I get?
"""

run_agent(prompt, config)

### 何が保存されたか？
ご覧のとおり、モデルはまだ私たちの好みや食事制限について何も把握していません。

pre/post モデルフックを使用したこの実装では、2つのメッセージが保存されました。ユーザーからの最初のメッセージと AI モデルからの応答の両方が AgentCore Memory に会話イベントとして保存されました。長期記憶が抽出されるまでに数秒かかる場合があるため、最初の試行で何も見つからない場合は数秒後に再試行してください。

これらのメッセージは、ファクトとユーザー設定の名前空間で AgentCore の長期記憶に抽出されました。実際、これまでに保存された内容をストア自体で確認できます：

In [ ]:
# Search our user preferences namespace
search_namespace = (actor_id, "preferences")
result = store.search(search_namespace, query="food", limit=3)
print(f"プリファレンス名前空間の結果: {result}")

### Store へのエージェントアクセス

**注意** - AgentCore Memory はこれらのイベントをバックグラウンドで処理するため、メモリが抽出されて長期記憶取得用に埋め込まれるまでに数秒かかる場合があります。

素晴らしい！会話の以前のメッセージに基づいて、長期記憶が名前空間に抽出されたことが確認できました。

次に、新しいセッションを開始して、夕食に何を作るべきかについての推薦を求めてみましょう。エージェントはストアを使用して抽出された長期記憶にアクセスし、ユーザーが気に入る推薦を行うことができます。

In [ ]:
config = {
    "configurable": {
        "thread_id": "session-2", # New session ID
        "actor_id": actor_id, # Same actor ID
    }
}

run_agent("Today's a new day, what should I make for dinner tonight?", config)

### まとめ

ご覧のとおり、エージェントはユーザー設定名前空間の検索から pre-model hook コンテキストを受け取り、ファクト名前空間の長期記憶を自分で検索して、ユーザーのために包括的な回答を作成することができました。

AgentCoreMemoryStore は非常に柔軟で、pre/post モデルフックやストア操作を持つツール自体など、さまざまな方法で実装できます。チェックポイント用の AgentCoreMemorySaver と一緒に使用することで、完全な会話状態と長期的なインサイトの両方を組み合わせて、複雑でインテリジェントなエージェントシステムを形成できます。